<div align="left">
<img align="left" src="./Data/logo-alinnco.png">
</div>

## Maestría en Inteligencia Artificial - Procesamiento de Lenguaje Natural
### Profesor: Dr. Gaddiel Desirena López
### Integrantes del equipo
* Guillermo Betanzos
* Luis Landeros
* Sergio Rojas
* Alberto Torres

#### Índice
[Objetivo](#objetivo)<br>
[Corpus](#corpus)<br>
[Tokenización](#tokenizacion)<br>
[Modelo](#modelo)<br>
[Modelo: Entrenamiento](#entrenamiento)<br>
[Modelo: Evaluación](#evaluacion)<br>
[Modelo: Pruebas](#pruebas)<br>
[Conclusiones](#conclusiones)<br>

### <a id='objetivo'>Objetivo</a>

El procesamiento del lenguaje natural tiene como objetivo fundamental lograr una comunicación maquina-humano similar a la comunicación humano-humano.

El objetivo principal del PLN es hacer que las máquinas comprendan textos no estructurados y exigan la información relevante de esos textos. Este se centra en el análisis de las comunicaciones humanas, en concreto de su propio lenguaje. Ante la gran cantidad de información en texto que generamos actualmente, surge la posibilidad de analizarla y aprovecharla. Las técnicas de PLN permiten extraer insights automáticamente de la información disponible en cualquier sector.

El objetivo de este proyecto, es el desarrollar una solución utilizando el Procesamiento de Lenguaje Natural para la predicción de palabras.

In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
tf.compat.v1.disable_eager_execution()
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

### <a id='corpus'>Corpus</a>

Hay pasos que puede seguir para preprocesar datos en lenguaje natural, de modo que el modelado que lleve a cabo en sentido descendente sea más preciso. Las opciones comunes de preprocesamiento de lenguaje natural incluyen:
* Tokenización
* Convertir caracteres a minúsculas
* Remoción de palabras vacías
* Eliminación de la puntuación
* Stemming
* Manejo de n-gramas

Para la elección del corpus del proyecto se consideró inicialmente el proyecto <a href="https://www.gutenberg.org/browse/languages/es" target="_blank">Gutenberg</a>, desafortunadamente los resultados obtenidos no fueron esperados ya que no se encontró congruencia alguna entre el texto proporcionado al modelo y la salida.

En vista de los resultados obtenidos procedimos a buscar corpus que fuesen más ricos en datos, tales como los corpus compuestos por Wikipedia, desafortunadamente las necesidades de recursos para procesar dichos corpus son demandantes en demasía.


<img src="./Data/memory.png">
<div align="center">Figura 1. Desbordamiento de memoria</div>

De tal suerte que dados los resultados en la elección del corpus hasta el momento se procedió a elegir uno que fuese relativamente más pequeño pero con contenido regionalizado.

In [2]:
path = 'lnr.txt'
text = open(path, encoding="utf-8").read().lower() + open("novelas.txt", encoding="utf-8").read().lower()
print('corpus length:', len(text))

corpus length: 423923


### <a id='tokenizacion'>Tokenización</a>

Antes de realizar una tarea de PLN hay que normalizar el texto, esto incluye 3 actividades:

1. Segmentación/tokenización de las palabras.
2. Normalización del formato de las plabras
3. Segmentación de las oraciones en el texto.

Algunos conceptos:

* Lema (Lemma): Palabras que comparten un tronco común, que hacen referencia al mismo concepto básico

    Ejemplo: gato, gatos, gata son palabras con el mismo lema.
    

* Forma de la palabra (Wordform): La forma completa de la palabra.

    Ejemplo: gato, gatos, gata son palabras con distinta forma (different wordform)
    

* Tipo (Type): Un elemento del vocabulario.

 
* Token: Una instancia de un tipo en un texto dado.


Se realiza la tokenización del corpus/dataset en cada palabra que lo compone excluyendo la presencia de caracteres especiales.

In [3]:
tokenizer = RegexpTokenizer(r'\b\w+\b')
words = tokenizer.tokenize(text)

NameError: name 'RegexpTokenizer' is not defined

In [ ]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

A través de la Ingeniería Funcional, la cuál es el proceso de creación de nuevas funciones a partir de datos sin procesar para aumentar el poder predictivo del algoritmo de aprendizaje, se definirá una longitud de palabra que representará el número de palabras anteriores las cuales determinarán la siguiente palabra. 

Asimismo definimos palabras previas para mantener cinco palabras anteriores y sus siguientes palabras correspondientes en la lista de palabras siguientes.

In [ ]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

In [ ]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [ ]:
print(X[0][0])

### <a id='modelo'>Modelo</a>

Para el modelo de predicción de palabra haremos uso de Redes Neuronales Recurrentes (RNN - Recurrent Neural Networks), en este caso muy particular la arquitectura de aprendizaje profundo LSTM (long short-term memory).

Las RNN son aplicables a cualquier dato que ocurre en una secuencia tales como las series de tiempo financieras, niveles de inventario, tráfico y el clima; esencialmente todo lo que involucre modelos de predicciones.



<img src="./Data/dl.png">
<div align="center">Figura 2. Deep Learning</div>

Las redes LSTM fueron introducidas por Sepp Hochreiter y Jürgen Schmidhuber en 1997, pero hoy en día se utilizan más ampliamente en aplicaciones de aprendizaje profundo de Procesamiento de Lenguaje Natural (PNL/NLP). 

La estructura básica de una capa LSTM es la misma que la de las capas recurrentes simples, reciben entradas de la secuencia de datos (por ejemplo, un token particular de un documento en lenguaje natural) y también reciben entradas del punto de tiempo anterior en la secuencia. 

La diferencia es que dentro de cada celda en una capa recurrente simple (por ejemplo, SimpleRNN () en Keras), encontrará una única función de activación de red neuronal, como una función tanh, que transforma las entradas de la celda RNN para generar su salida. 

En contraste, las celdas de una capa LSTM contienen una estructura mucho más compleja, poseen conexiones de retroalimentación a diferencia de las redes más tradicionales "feed forward".

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

### <a id='entrenamiento'>Modelo: Entrenamiento</a>

Como parte de la implementación del modelo se decidió por el optimizador RMSprop (root mean square propagation) el cuál fue desarrollado por Geoff Hinton aproximadamente al mismo tiempo que AdaDelta.

Funciona de manera similar excepto que retiene el parámetro de tasa de aprendizaje η. Tanto RMSProp como AdaDelta implican un hiperparámetro adicional ρ (rho), o tasa de decaimiento, que es análogo al valor β del impulso y que guía el tamaño de la ventana para la media móvil. Los valores recomendados para los hiperparámetros son ρ = 0,95 para ambos optimizadores y el ajuste η = 0,001 para RMSProp.

In [ ]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=50, shuffle=True).history

In [ ]:
#unique_words = np.unique(words)
#unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [ ]:
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')Guillermo

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x

In [ ]:
prepare_input("la niña era")

In [ ]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [ ]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [ ]:
q = "no funciona"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

### <a id='conclusiones'>Conclusiones</a>

Con el propósito de dar a conocer el estado actual del Procesamiento del Lenguaje Natural se han definido, de forma muy concisa, los principales conceptos y técnicas asociados a esta disciplina, que además se ha desarrollado a lo largo de este proyecto escolar.

Así mismo, se ha comprobado que el PLN es una disciplina viva y en pleno desarrollo, con multitud de retos que superar fruto de la ambigüedad subyacente al lenguaje natural. El empleo del lenguaje le permite al hombre trasmitir sus conocimientos, sentimientos, sensaciones, emociones, y estados de ánimo, por lo que han sido varios los sistemas informáticos inteligentes que se han desarrollado y continuan en evolución continua que emplean el procesamiento del lenguaje natural, para una mayor comprensión y mejoría en el ámbito de las tecnologías de información.